In [1]:
print('test')

test


In [2]:
import sagemaker , boto3

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\bored\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
bucket="prahaladreddy45"

In [4]:
s3=boto3.client('s3')

In [5]:
items=s3.list_objects_v2(
    Bucket=bucket)

In [6]:
items

{'ResponseMetadata': {'RequestId': '5VVQPESHY86J3ZV7',
  'HostId': 'XyIWaO5V7An6s4XTsp0BW2KVmO0S5cJbf4Ii+Fm0L5SkdBdrj4SJ6ny/DwtPAlgyUn6JpPbARQU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'XyIWaO5V7An6s4XTsp0BW2KVmO0S5cJbf4Ii+Fm0L5SkdBdrj4SJ6ny/DwtPAlgyUn6JpPbARQU=',
   'x-amz-request-id': '5VVQPESHY86J3ZV7',
   'date': 'Sat, 21 Jun 2025 06:35:10 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'data/iris_infer.csv',
   'LastModified': datetime.datetime(2025, 6, 21, 6, 32, 18, tzinfo=tzutc()),
   'ETag': '"10b2df6ea2d26d9a74cc6893383af7fa"',
   'ChecksumAlgorithm': ['CRC64NVME'],
   'ChecksumType': 'FULL_OBJECT',
   'Size': 850,
   'StorageClass': 'STANDARD'},
  {'Key': 'data/iris_test.csv',
   'LastModified': datetime.datetime(2025, 6, 21, 6, 32, 24, tzinfo=tzutc()),
   'ETag': '"a47b2450ef5c8d408bddea8e4f28f

In [7]:
items=items['Contents']
for item in items:
    print(item['Key'])

data/iris_infer.csv
data/iris_test.csv
data/iris_train.csv


In [9]:
iam=boto3.client('iam')
role_name = 'SageMakerExecutionRole'
try:
    role = iam.get_role(RoleName=role_name)
    print(f"Role {role_name} already exists.")
except iam.exceptions.NoSuchEntityException:
    role = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument='''{
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "sagemaker.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }'''
    )
    print(f"Created role {role_name}.")

Role SageMakerExecutionRole already exists.


In [10]:
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

{'ResponseMetadata': {'RequestId': '4b0d631e-8586-49d5-a82b-93e172cca4ea',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 21 Jun 2025 06:37:30 GMT',
   'x-amzn-requestid': '4b0d631e-8586-49d5-a82b-93e172cca4ea',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [12]:
role_arn=role['Role']['Arn']

In [ ]:
!aws s3 

In [40]:
training_data_path = f's3://{bucket}/data/iris_train.csv'
validation_data_path = f's3://{bucket}/data/iris_test.csv'
inference_data_path = f's3://{bucket}/data/iris_inference.csv'

In [44]:
training_data_path

's3://prahaladreddy45/data/iris_train.csv'

In [49]:
model_path=f's3://{bucket}/model/iris_model'

In [50]:
model_path

's3://prahaladreddy45/model/iris_model'

In [16]:
from sagemaker import TrainingInput

In [42]:
train_ip=TrainingInput(training_data_path,content_type='csv')
validation_ip=TrainingInput(validation_data_path,content_type='csv')

In [25]:
region = sagemaker.Session().boto_region_name

In [26]:
model_image=sagemaker.image_uris.retrieve('xgboost',region,"latest")

In [27]:
base_job_name = "iris-xgboost-"

In [51]:
xgboost_estimator=sagemaker.estimator.Estimator(
  image_uri=model_image,
  role=role_arn,
  instance_count=1,
  instance_type='ml.m5.large',
  base_job_name=base_job_name,
  output_path=model_path,
  sagemaker_session=sagemaker.Session(),
  volume_size=5,
)

In [52]:
xgboost_estimator.set_hyperparameters(
    num_class=3, max_depth=5, num_round=10, objective="multi:softmax",
)

In [33]:
from datetime import datetime

In [53]:
new_job=base_job_name + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [54]:
xgboost_estimator.fit(
  {'train':train_ip, 'validation':validation_ip},
  job_name=new_job,
  wait=True
)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: iris-xgboost-2025-06-21-12-54-34


2025-06-21 07:24:40 Starting - Starting the training job...
2025-06-21 07:25:17 Downloading - Downloading input data......
2025-06-21 07:26:02 Downloading - Downloading the training image.Arguments: train
[2025-06-21:07:26:41:INFO] Running standalone xgboost training.
[2025-06-21:07:26:41:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 268.57mb
[2025-06-21:07:26:41:INFO] Determined delimiter of CSV input is ','
[07:26:41] S3DistributionType set as FullyReplicated
[07:26:41] 100x4 matrix with 400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-06-21:07:26:41:INFO] Determined delimiter of CSV input is ','
[07:26:41] S3DistributionType set as FullyReplicated
[07:26:41] 50x4 matrix with 200 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[07:26:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[07:26:41] src/tree/

In [ ]:
import botocore

try:
  s3.head_bucket(Bucket=bucket)
  print(f"Bucket '{bucket}' already exists.")
except botocore.exceptions.ClientError:
  s3.create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': region})
  print(f"Bucket '{bucket}' created.")

Bucket 'prahaladreddy45' already exists.


In [48]:
import boto3
s3 = boto3.client('s3')
print(s3.get_bucket_location(Bucket=bucket))


{'ResponseMetadata': {'RequestId': '3ARB65BQ2WX2S6BK', 'HostId': 'GFcgMtzJFsZw1LFpwwbOClPV4Ds1crUGFRn4UkwB+uD12qgtDhgD1s6yyvrCxJYMF0YhOcHPepM=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'GFcgMtzJFsZw1LFpwwbOClPV4Ds1crUGFRn4UkwB+uD12qgtDhgD1s6yyvrCxJYMF0YhOcHPepM=', 'x-amz-request-id': '3ARB65BQ2WX2S6BK', 'date': 'Sat, 21 Jun 2025 07:19:57 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'LocationConstraint': None}


In [55]:
new_job

'iris-xgboost-2025-06-21-12-54-34'

In [58]:
xgb_model=sagemaker.estimator.Estimator.attach(new_job)


2025-06-21 07:27:01 Starting - Preparing the instances for training
2025-06-21 07:27:01 Downloading - Downloading the training image
2025-06-21 07:27:01 Training - Training image download completed. Training in progress.
2025-06-21 07:27:01 Uploading - Uploading generated training model
2025-06-21 07:27:01 Completed - Training job completed


In [62]:
xgb_predictor = xgb_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    serializer=sagemaker.serializers.CSVSerializer(),
)

INFO:sagemaker:Creating model with name: iris-xgboost-2025-06-21-12-54-34-2025-06-21-08-11-39-535
INFO:sagemaker:Creating endpoint-config with name iris-xgboost-2025-06-21-12-54-34-2025-06-21-08-11-39-535
INFO:sagemaker:Creating endpoint with name iris-xgboost-2025-06-21-12-54-34-2025-06-21-08-11-39-535


------!

In [64]:
endpoint_nmae=xgb_predictor.endpoint_name

In [65]:
sage_runtime=boto3.client('sagemaker-runtime')

In [66]:
response=sage_runtime.invoke_endpoint(
    EndpointName=endpoint_nmae,
    Body='5.1,3.5,1.4,0.2',
    ContentType='text/csv'
)

In [67]:
response

{'ResponseMetadata': {'RequestId': '955156f6-e97e-4b4d-affd-6607d894e6bf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '955156f6-e97e-4b4d-affd-6607d894e6bf',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Sat, 21 Jun 2025 08:17:38 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '3',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x1f74e47f430>}

In [68]:
response['Body'].read().decode('utf-8')

'0.0'

In [70]:
batch_op = "s3://{}/{}".format(bucket,"batch_transform")

In [71]:
transformer = xgb_model.transformer(
    instance_count=1, instance_type="ml.m4.xlarge", output_path=batch_op
)

INFO:sagemaker:Creating model with name: iris-xgboost-2025-06-21-12-54-34-2025-06-21-08-20-21-603


In [72]:
from sagemaker.transformer import Transformer

In [77]:
batch_ip='s3://prahaladreddy45/data/iris_infer.csv'

In [78]:
transformer.transform(
    data=batch_ip, data_type="S3Prefix", content_type="text/csv", split_type="Line"
)
transformer.wait()

INFO:sagemaker:Creating transform job with name: iris-xgboost-2025-06-21-12-54-34-2025-06-21-08-23-00-432


.............................Arguments: serve
[2025-06-21 08:29:12 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2025-06-21 08:29:12 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2025-06-21 08:29:12 +0000] [1] [INFO] Using worker: gevent
[2025-06-21 08:29:12 +0000] [21] [INFO] Booting worker with pid: 21
[2025-06-21 08:29:12 +0000] [22] [INFO] Booting worker with pid: 22
[2025-06-21 08:29:12 +0000] [23] [INFO] Booting worker with pid: 23
[2025-06-21 08:29:12 +0000] [24] [INFO] Booting worker with pid: 24
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py)',

In [79]:
!aws s3 ls {bucket}/batch_transform/ --recursive

2025-06-21 13:59:17        200 batch_transform/iris_infer.csv.out


In [80]:
smkr_client=boto3.client('sagemaker')

In [81]:
smkr_client.delete_endpoint(EndpointName=endpoint_nmae)

{'ResponseMetadata': {'RequestId': '0854990d-3a80-4424-a6d2-9caea483d16e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0854990d-3a80-4424-a6d2-9caea483d16e',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 21 Jun 2025 08:34:12 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}